In [10]:
import cv2
import numpy as np
import os

def preprocess_image(image_path, target_size=(224, 224)):
    """Preprocesses the input gesture image."""
    try:
        image = cv2.imread(image_path)
        if image is None:
            raise FileNotFoundError(f"Failed to read image from path: {image_path}")
        image = cv2.resize(image, target_size)
        return image
    except Exception as e:
        print(f"Error preprocessing image: {e}")
        return None

def preprocess_video(video_path, target_size=(224, 224)):
    """Extracts frames from the test video and preprocesses them."""
    try:
        frames = []
        cap = cv2.VideoCapture(video_path)
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, target_size)
            frames.append(frame)
        cap.release()
        return frames
    except Exception as e:
        print(f"Error preprocessing video: {e}")
        return None

def detect_gesture(gesture_representation, test_frames, threshold=0.7):
    """Detects the specified gesture in the test frames."""
    try:
        detected_frames = []
        for frame in test_frames:
            # Perform feature-based matching
            orb = cv2.ORB_create()
            kp1, des1 = orb.detectAndCompute(gesture_representation, None)
            kp2, des2 = orb.detectAndCompute(frame, None)
            
            # Brute force matching
            bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
            matches = bf.match(des1, des2)
            
            # Apply ratio test
            good_matches = [m for m in matches if m.distance < threshold]
            
            if len(good_matches) > 10:  # Adjust threshold as needed
                detected_frames.append(frame)
        return detected_frames
    except Exception as e:
        print(f"Error detecting gesture: {e}")
        return None

def annotate_frames(frames):
    """Annotates the frames with 'DETECTED' in bright green."""
    try:
        for frame in frames:
            cv2.putText(frame, 'DETECTED', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        return frames
    except Exception as e:
        print(f"Error annotating frames: {e}")
        return None

def save_video(frames, output_video_path):
    """Saves the annotated frames as a video."""
    try:
        if not frames:
            raise ValueError("No frames to save.")
        
        height, width, _ = frames[0].shape
        if height == 0 or width == 0:
            raise ValueError("Invalid frame dimensions.")
        
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (width, height))
        for frame in frames:
            out.write(frame)
        out.release()
        print("Annotated video saved at:", output_video_path)
    except Exception as e:
        print(f"Error saving video: {e}")

# Paths to input data
test_video_path = 'C:\\Users\\dell\\Pictures\\Download\\data.mp4'
gesture_image_path = "C:\\Users\\dell\\Desktop\\78b54d5812ca86abb6a6784b0a1f3755.jpg"

# Preprocess the gesture representation
gesture_representation = preprocess_image(gesture_image_path)
if gesture_representation is None:
    exit()

# Preprocess the test video
test_frames = preprocess_video(test_video_path)
if test_frames is None:
    exit()

# Detect the gesture in the test frames
detected_frames = detect_gesture(gesture_representation, test_frames)

# Annotate the detected frames and save the video
if detected_frames:
    annotated_frames = annotate_frames(detected_frames)
    if annotated_frames:
        # Path to save the annotated video
        output_video_path = 'output/annotated_video.mp4'
        save_video(annotated_frames, output_video_path)
    else:
        print("Failed to annotate frames.")
else:
    print("No frames were detected containing the specified gesture.")


No frames were detected containing the specified gesture.
